In [16]:
import gmsh
import numpy as np

In [17]:
gmsh.initialize()
gmsh.model.add("my_model")

In [18]:
# 读取numpy数组


In [19]:
# n行m列的矩阵，代表dem数据
# zlist = np.load("terrain.npy")
zlist = np.array([[0.7, 1.1, 1.2, 1.3],
                  [1.1, 0.7, 0.8, 1.2],
                  [1.2, 0.9, 0.7, 1.1]])
x_dist = 11252.55935707817 # x方向的总间距
y_dist = 9608.555047642712 # y方向的总间距
lc1 = 50; # 近地面边界层网格尺寸
lc2 = 500; # 远离地面边界层网格尺寸
zMax = zlist.max() + 2000; # 最大高度
zLayer = 10; # 边界层厚度

In [20]:
dx = x_dist/zlist.shape[1] # x方向的间距
dy = y_dist/zlist.shape[0] # y方向的间距

In [21]:
zlist = np.flipud(zlist)
# 创建两个空的数组，形状和zlist一样n行m列
# 储存上下面上每个格点的tag序号
plist_lower = np.zeros_like(zlist).astype(int)
plist_upper = np.zeros_like(zlist).astype(int)
# 创建代表底面上平行于x轴的线段的tag序号的数组
# 形状为n行m-1列
llist_x_lower = np.zeros((zlist.shape[0], zlist.shape[1] - 1)).astype(int)
# 创建代表底面上平行于y轴的线段的tag序号的数组
# 形状为n-1行m列
llist_y_lower = np.zeros((zlist.shape[0] - 1, zlist.shape[1])).astype(int)
# 创建代表上面上平行于x轴的线段的tag序号的数组
# 形状为n行m-1列
llist_x_upper = np.zeros((zlist.shape[0], zlist.shape[1] - 1)).astype(int)
# 创建代表上面上平行于y轴的线段的tag序号的数组
# 形状为n-1行m列
llist_y_upper = np.zeros((zlist.shape[0] - 1, zlist.shape[1])).astype(int)
# 创建代表平行于z方向的线段的tag序号的数组
# 形状为n行m列
llist_z = np.zeros_like(zlist).astype(int)
# 创建代表底面上的线环的tag序号的数组
# 形状为n-1行m-1列
llist_loop_lower = np.zeros((zlist.shape[0] - 1, zlist.shape[1] - 1)).astype(int)
# 创建代表底面上的面的tag序号的数组
# 形状为n-1行m-1列
slist_lower = np.zeros((zlist.shape[0] - 1, zlist.shape[1] - 1)).astype(int)

In [22]:
factory = gmsh.model.occ

In [23]:
for n_row, row in enumerate(zlist):
    for n_col, z in enumerate(row):
        plist_lower[n_row][n_col] = factory.addPoint(n_col*dx, n_row*dy, z, lc1)
        plist_upper[n_row][n_col] = factory.addPoint(n_col*dx, n_row*dy, zMax, lc2)

# 平行于x轴的线
for n_row, row in enumerate(llist_x_lower):
    for n_col, p_index in enumerate(row):
        llist_x_lower[n_row][n_col] = factory.addLine(plist_lower[n_row][n_col], plist_lower[n_row][n_col + 1])
        llist_x_upper[n_row][n_col] = factory.addLine(plist_upper[n_row][n_col], plist_upper[n_row][n_col + 1])

# 平行于y轴的线
for n_row, row in enumerate(llist_y_lower):
    for n_col, p_index in enumerate(row):
        llist_y_lower[n_row][n_col] = factory.addLine(plist_lower[n_row][n_col], plist_lower[n_row + 1][n_col])
        llist_y_upper[n_row][n_col] = factory.addLine(plist_upper[n_row][n_col], plist_upper[n_row + 1][n_col])

# 平行于z轴的线
for n_row, row in enumerate(llist_z):
    for n_col, p_index in enumerate(row):
        llist_z[n_row][n_col] = factory.addLine(plist_lower[n_row][n_col], plist_upper[n_row][n_col])

In [24]:
# 创建底面的线环和面
for n_row, row in enumerate(llist_loop_lower):
    for n_col, p_index in enumerate(row):
        llist_loop_lower[n_row][n_col] = factory.addCurveLoop([llist_x_lower[n_row][n_col],
                                                               llist_y_lower[n_row][n_col + 1],
                                                               -llist_x_lower[n_row + 1][n_col],
                                                               -llist_y_lower[n_row][n_col]])
        slist_lower[n_row][n_col] = factory.addSurfaceFilling(llist_loop_lower[n_row][n_col])

In [25]:
# 创建front面的线环和面
curve_loop_front = np.concatenate((llist_x_lower[0], [llist_z[0][-1]], -llist_x_upper[0][::-1], [-llist_z[0][0]])).astype(int).tolist()
s_front = factory.addPlaneSurface([factory.addCurveLoop(curve_loop_front)])

# 创建back面的线环和面
curve_loop_back = np.concatenate((llist_x_lower[-1], [llist_z[-1][-1]], -llist_x_upper[-1][::-1], [-llist_z[-1][0]])).astype(int).tolist()
s_back = factory.addPlaneSurface([factory.addCurveLoop(curve_loop_back)])

# 创建left面的线环和面
curve_loop_left = np.concatenate((llist_y_lower[:, 0], [llist_z[-1][0]], -llist_y_upper[:, 0][::-1], [-llist_z[0][0]])).astype(int).tolist()
s_left = factory.addPlaneSurface([factory.addCurveLoop(curve_loop_left)])

# 创建right面的线环和面
curve_loop_right = np.concatenate((llist_y_lower[:, -1], [llist_z[-1][-1]], -llist_y_upper[:, -1][::-1], [-llist_z[0][-1]])).astype(int).tolist()
s_right = factory.addPlaneSurface([factory.addCurveLoop(curve_loop_right)])

# 创建top面的线环和面
curve_loop_top = np.concatenate((llist_x_upper[0], 
                                 llist_y_upper[:, -1],
                                 -llist_x_upper[-1][::-1],
                                 -llist_y_upper[:, 0][::-1])).astype(int).tolist()
s_top = factory.addPlaneSurface([factory.addCurveLoop(curve_loop_top)])

In [26]:
# 创建体
surface_loop = np.concatenate((slist_lower.flatten(),
                [s_front, s_back, s_left, s_right, s_top])).astype(int).tolist()
v = factory.addVolume([factory.addSurfaceLoop(surface_loop)])

In [27]:
# 底面向上进行拉伸创建边界层网格
surface_to_extrude = [(2, s_index) for s_index in slist_lower.flatten()]
out_dim_tags = factory.extrude(
    surface_to_extrude,
    0, 0, zLayer,  # extrusion along z-axis
    numElements=[1,1,1,1,1], 
    heights=[0.1343797033,0.2956353472,0.4891421200,0.7213502473,1], #todo: 编写边界层厚度计算程序
    recombine=True
)

In [28]:
extrude_v_list = [tag for tag in out_dim_tags if tag[0] == 3]
v_cut = factory.cut([(3, v)], extrude_v_list, removeTool=False)

In [29]:
factory.synchronize()

In [30]:
gmsh.option.setNumber("Mesh.MshFileVersion", 2.2)
gmsh.model.addPhysicalGroup(2, slist_lower.flatten().tolist(), name="terrain")
gmsh.model.addPhysicalGroup(2, [s_front], name="front")
gmsh.model.addPhysicalGroup(2, [s_right], name="right")
gmsh.model.addPhysicalGroup(2, [s_back], name="back")
gmsh.model.addPhysicalGroup(2, [s_left], name="left")
gmsh.model.addPhysicalGroup(2, [s_top], name="upperWall")
gmsh.model.addPhysicalGroup(3, [v]+[tag[1] for tag in extrude_v_list], name="interfluid")
gmsh.model.mesh.generate(3)
gmsh.write("mesh.msh")
# gmsh.fltk.run()

Exception: Could not find extruded node (9227.098672804099, 2092.52976593108, 2129.333333333333) in surface 29488

In [31]:
gmsh.fltk.run()

In [ ]:
gmsh.finalize()